<a href="https://colab.research.google.com/github/tmdang1101/amazon_product_recommender_system/blob/main/Model_2_2_Content_Based_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries #

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt

import gzip
import json

In [ ]:
# Connects this notebook to Google Drive
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

pathname = "/content/drive/My Drive/Recommender System/"
os.chdir(pathname)

Mounted at /content/drive


# Data Pre-Processing #

## Metadata Dataset ##

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getMetadataDF(path):
    i = 0
    df = {}
    for d in parse(path):
        # new_d = []
        # new_d.append(d['title'])
        # new_d.append(d['also_buy'])
        # new_d.append(d['also_view'])
        # new_d.append(d['similar_item'])
        # new_d.append(d['asin'])
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getMetadataDF('/content/drive/My Drive/Recommender System/Data/meta_Movies_and_TV.json.gz')

In [ ]:
df.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Movies & TV, Movies]",,[],,Understanding Seizures and Epilepsy,[],,,[],"886,503 in Movies & TV (",[],Movies & TV,,,,0000695009,[],[],NaN
1,"[Movies & TV, Movies]",,[],,Spirit Led&mdash;Moving By Grace In The Holy S...,[],,,[],"342,688 in Movies & TV (",[],Movies & TV,,,,0000791156,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Movies & TV, Movies]",,[Disc 1: Flour Power (Scones; Shortcakes; Sout...,,My Fair Pastry (Good Eats Vol. 9),[],,Alton Brown,[],"370,026 in Movies & TV (",[],Movies & TV,,,,0000143529,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,"[Movies & TV, Movies]",,[Barefoot Contessa Volume 2: On these three di...,,"Barefoot Contessa (with Ina Garten), Entertain...","[B002I5GNW4, B005WXPVMM, B009UY3W8O, B00N27ID1...",,Ina Garten,[],"342,914 in Movies & TV (","[B002I5GNW4, 0804187045, B009UY3W8O, 060960219...",Movies & TV,,,$74.95,0000143588,[],[],NaN
4,"[Movies & TV, Movies]",,[Rise and Swine (Good Eats Vol. 7) includes bo...,,Rise and Swine (Good Eats Vol. 7),"[B000P1CKES, B000NR4CRM]",,Alton Brown,[],"351,684 in Movies & TV (",[B0015SVNXY],Movies & TV,,,,0000143502,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [ ]:
df.shape

(203766, 19)

## Creating Tags Feature ##

In [ ]:
content = df[['asin', 'title', 'description', 'brand']]

In [ ]:
content.head()

,asin,title,description,brand
0,0000695009,Understanding Seizures and Epilepsy,[],
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,[],
2,0000143529,My Fair Pastry (Good Eats Vol. 9),[Disc 1: Flour Power (Scones; Shortcakes; Sout...,Alton Brown
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...",[Barefoot Contessa Volume 2: On these three di...,Ina Garten
4,0000143502,Rise and Swine (Good Eats Vol. 7),[Rise and Swine (Good Eats Vol. 7) includes bo...,Alton Brown


In [ ]:
content.shape

(203766, 4)

In [ ]:
content['description'] = content['description'].apply(lambda x : x[0].split() if len(x) > 0 else [])

<ipython-input-9-84caee758e70>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['description'] = content['description'].apply(lambda x : x[0].split() if len(x) > 0 else [])


In [ ]:
content.head()

,asin,title,description,brand
0,0000695009,Understanding Seizures and Epilepsy,[],
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,[],
2,0000143529,My Fair Pastry (Good Eats Vol. 9),"[Disc, 1:, Flour, Power, (Scones;, Shortcakes;...",Alton Brown
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...","[Barefoot, Contessa, Volume, 2:, On, these, th...",Ina Garten
4,0000143502,Rise and Swine (Good Eats Vol. 7),"[Rise, and, Swine, (Good, Eats, Vol., 7), incl...",Alton Brown


In [ ]:
content['brand'] = content['brand'].apply(lambda x : [x.replace(" ", "")] if len(x) > 0 else [])

<ipython-input-11-05ccf62f638b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['brand'] = content['brand'].apply(lambda x : [x.replace(" ", "")] if len(x) > 0 else [])


In [ ]:
content.head()

,asin,title,description,brand
0,0000695009,Understanding Seizures and Epilepsy,[],[]
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,[],[]
2,0000143529,My Fair Pastry (Good Eats Vol. 9),"[Disc, 1:, Flour, Power, (Scones;, Shortcakes;...",[AltonBrown]
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...","[Barefoot, Contessa, Volume, 2:, On, these, th...",[InaGarten]
4,0000143502,Rise and Swine (Good Eats Vol. 7),"[Rise, and, Swine, (Good, Eats, Vol., 7), incl...",[AltonBrown]


In [ ]:
df.rename(columns={'asin':'movie_id'}, inplace=True)

In [ ]:
content.rename(columns={'asin':'movie_id'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
content.head()

,movie_id,title,description,brand
0,0000695009,Understanding Seizures and Epilepsy,[],[]
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,[],[]
2,0000143529,My Fair Pastry (Good Eats Vol. 9),"[Disc, 1:, Flour, Power, (Scones;, Shortcakes;...",[AltonBrown]
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...","[Barefoot, Contessa, Volume, 2:, On, these, th...",[InaGarten]
4,0000143502,Rise and Swine (Good Eats Vol. 7),"[Rise, and, Swine, (Good, Eats, Vol., 7), incl...",[AltonBrown]


In [ ]:
content['tags'] = content['description'] + content['brand']

<ipython-input-16-06adfb773ab3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['tags'] = content['description'] + content['brand']


In [ ]:
content = content[['movie_id','title','tags']]

In [ ]:
content.head()

,movie_id,title,tags
0,0000695009,Understanding Seizures and Epilepsy,[]
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,[]
2,0000143529,My Fair Pastry (Good Eats Vol. 9),"[Disc, 1:, Flour, Power, (Scones;, Shortcakes;..."
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...","[Barefoot, Contessa, Volume, 2:, On, these, th..."
4,0000143502,Rise and Swine (Good Eats Vol. 7),"[Rise, and, Swine, (Good, Eats, Vol., 7), incl..."


In [ ]:
content['tags'] = content['tags'].apply(lambda x : " ".join(x))

In [ ]:
content.head()

,movie_id,title,tags
0,0000695009,Understanding Seizures and Epilepsy,
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,
2,0000143529,My Fair Pastry (Good Eats Vol. 9),Disc 1: Flour Power (Scones; Shortcakes; South...
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...",Barefoot Contessa Volume 2: On these three dis...
4,0000143502,Rise and Swine (Good Eats Vol. 7),Rise and Swine (Good Eats Vol. 7) includes bon...


In [ ]:
content['tags'] = content['tags'].apply(lambda x : x.lower())

In [ ]:
content.head()

,movie_id,title,tags
0,0000695009,Understanding Seizures and Epilepsy,
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,
2,0000143529,My Fair Pastry (Good Eats Vol. 9),disc 1: flour power (scones; shortcakes; south...
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...",barefoot contessa volume 2: on these three dis...
4,0000143502,Rise and Swine (Good Eats Vol. 7),rise and swine (good eats vol. 7) includes bon...


In [ ]:
# combine words belonging to the same family
import nltk

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [ ]:
content['tags'] = content['tags'].apply(stem)

In [ ]:
content.head()

,movie_id,title,tags
0,0000695009,Understanding Seizures and Epilepsy,
1,0000791156,Spirit Led&mdash;Moving By Grace In The Holy S...,
2,0000143529,My Fair Pastry (Good Eats Vol. 9),disc 1: flour power (scones; shortcakes; south...
3,0000143588,"Barefoot Contessa (with Ina Garten), Entertain...",barefoot contessa volum 2: on these three disc...
4,0000143502,Rise and Swine (Good Eats Vol. 7),rise and swine (good eat vol. 7) includ bonu f...


# Similarity Model #

In [ ]:
# convert a collection of text documents to a matrix of TF-IDF
# account for stop words (i.e is, are, and, of, to, from, etc)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 5000, stop_words = 'english')
vectors = vectorizer.fit_transform(content['tags']).toarray()

In [ ]:
vectors[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, k, metric='cosine', show_distance=False):
    """
    Finds k-nearest neighbours for a given movie id.
    
    Args:
        movie_id: id of the movie of interest
        X: user-item utility matrix
        k: number of similar movies to retrieve
        metric: distance metric for kNN calculations
    
    Returns:
        list of k similar movie ID's
    """
    neighbour_ids = []
    
    movie_ind = list(df['movie_id']).index(movie_id)
    movie_vec = vectors[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(vectors)
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(content.loc[n]['movie_id'])
    neighbour_ids.pop(0)
    return neighbour_ids

# Recommendation #

In [ ]:
#movie_id = 'B00IT70F3S'     # Avengers: Age of Ultron
#movie_id = 'B00LYYUTEI'    # The Dark Knight
movie_id = 'B00005JMAH'     # Harry Potter and the Prisoner of Azkaban

similar_ids = find_similar_movies(movie_id, k=20)

print(f"Because you watched {movie_id}:\n")
for id in similar_ids:
    print(f"{id}\n{content.loc[content['movie_id'] == id]['title']}\n")

Because you watched B00005JMAH:

B00005JMAH
26876    Harry Potter and the Prisoner of Azkaban
48803    Harry Potter and the Prisoner of Azkaban
Name: title, dtype: object

B000E6UZZK
79471    Harry Potter - Years 1-4: (Harry Potter and th...
Name: title, dtype: object

B0002TV2WO
42703    Harry Potter Years 1-3
64630    Harry Potter Years 1-3
Name: title, dtype: object

B0002TV2WO
42703    Harry Potter Years 1-3
64630    Harry Potter Years 1-3
Name: title, dtype: object

B004KSRML8
145367    Harry Potter Collection - Years 1-7 Part 1 Reg...
Name: title, dtype: object

B00KP073LO
185225    Harry Potter and the Philosopher's Stone [Wide...
Name: title, dtype: object

B0000633SA
29783    Discovering the Real World of Harry Potter
51710    Discovering the Real World of Harry Potter
Name: title, dtype: object

B0000633SA
29783    Discovering the Real World of Harry Potter
51710    Discovering the Real World of Harry Potter
Name: title, dtype: object

7799737832
19362    Harry Potter 1-7 Col